<p align="center">  <img src="BeyondQuantum_Banner_Research_Projects_2025.png" alt="BeyondQuantum Research Banner"></p>

# Discrete Fourier Transform-based Quantum Circuit for Modular Exponentiation in Shor's Algorithm

**Author**: Roman Bagdasarian

**Date**: May 27th, 2025

**Acknowledgments**: This work was supported by [BlueQubit](https://www.bluequbit.io) – Quantum Software company based in Bay Area, California, founded by Hayk Tepanyan and Hrant Gharibyan.

## Overview

This notebook implements the discretee Fourier transform-based modular multiplication circuit from Patoary *et al.* (2025) and plugs it into Shor's algorithm.

## 🚀 Environment Setup

Install or upgrade the required dependencies.

In [ ]:
# Upgrade pip to the latest version
!pip install --upgrade pip

# Install Qiskit 2.0.0
!pip install qiskit==2.0.0

# Install the BlueQubit Software Development Kit
!pip install bluequbit==0.12.2b1

# Install NumPy
!pip install numpy==2.2.5

# Install Matplotlib
!pip install matplotlib==3.10.3

### 📦 Import Modules

In [ ]:
# Qiskit
import qiskit
from qiskit                 import QuantumRegister, ClassicalRegister, QuantumCircuit, transpile
from qiskit.circuit.library import QFT, CHGate

# BlueQubit Software Development Kit
import bluequbit

# NumPy
import numpy                as np

# Matplotlib
import matplotlib.pyplot    as plt

# Mathematics
import math
from math       import gcd, ceil, log2
from fractions  import Fraction

## Discrete Fourier Transform-based Modular Multiplication Circuit

Patoary et al. show that
$$
[U_A]_{nm} = \frac{1}{N} \sum_{k=0}^{N-1} \mathrm{e}^{\frac{2\pi i k (n - A m)}{N}} \;\equiv\; [F_N^{-1}G_N]_{nm} \tag{4}
$$
where
$$
[F_N]_{nm} = \frac{1}{\sqrt{N}} \mathrm{e}^{-\frac{2\pi i n m}{N}} \tag{5}
$$
$$
[G_N]_{nm} = \frac{1}{\sqrt{N}} \mathrm{e}^{-\frac{2\pi i A n m}{N}} \tag{6}
$$

### Outline of the DFT-based Modular Multiplication Circuit

1. **$S$ gate**

Prepare ancilla in 
$$
\lvert\Psi_N\rangle = \frac{1}{\sqrt{N}}\sum_{l=0}^{N-1}\lvert l\rangle \tag{7}
$$
via recursive single-qubit rotations.

2. **$V_A$ gate**

Two-register conrolled-phase
$$
V_a|m\rangle_1|n\rangle_2 \mapsto |m\rangle_1 e^{\frac{2\pi iAmn}{N}}|n\rangle_2 \tag{7}
$$

3. **Operator $K_G$**

Reset ancilla and swap by:
- $F_A^\dagger$ (inverse scalled-DFT) on register $m$
- Controlled shifts $W^{2^k}$ onto ancilla
- Hadamard gates on phase qubits
- Swap $m\leftrightarrow a$

4. **$S$ gate**

Re-prepare $\lvert\Psi_N\rangle$ (Eq. 7).

5. **$V_1$ gate**

$V_A$ gate with $A = 1$ to implement $F_N^{-1}$

6. **Operator $K_F$**

Reset via $F_2^L{}^\dagger$, controlled shifts $W^{2^k}$, and swap.

The full sequence
$$
\lvert m\rangle\lvert 0\rangle
\;\xrightarrow{S\to V_A\to K_G\to S\to V_1\to K_F}\;
\lvert mA\bmod N\rangle\lvert 0\rangle
$$
implements
$$
[U_A]_{nm} \;\equiv\; [F_N^{-1}G_N]_{nm} \tag{4}
$$

### S gate

Prepare the equal superposition state
$$
|Ψ_N\rangle = \frac1{\sqrt N}\sum_{m=0}^{N-1}|l⟩
$$
on an L-qubit register `reg` initialized in $\lvert0\rangle_L$.  


In [ ]:
def S(qc: QuantumCircuit,
      reg: QuantumRegister,
      N: int
) -> None:
    L = len(reg)
    N_bar = N
    # Big-endian bit list of N
    bits = list(map(int, format(N, f'0{L}b')))
    for i in range(L):
        if bits[i] == 1:
            theta = np.arctan(np.sqrt(N_bar / 2**(L - i  -1) - 1))
            qc.ry(2 * theta, reg[i])
            for j in range(i + 1, L):
                qc.x(reg[i])
                qc.append(CHGate(), [reg[i], reg[j]])
                qc.x(reg[i])
            N_bar -= 2**(L - i - 1)

### $V_A$ gate

Two-register phase operator
$$
V_a|m\rangle_1|n\rangle_2 \mapsto |m\rangle_1 e^{\frac{2\pi iAmn}{N}}|n\rangle_2 \tag{9}
$$
which implements
$$
[G_N]_{nm} = \frac{1}{\sqrt{N}} \mathrm{e}^{-\frac{2\pi i A n m}{N}} \tag{6}
$$

In [ ]:
def V(qc: QuantumCircuit,
      reg_m: QuantumRegister,
      reg_n: QuantumRegister,
      A: int,
      N: int
)-> None:
    L = len(reg_m)
    # For each bit k of m and bit j of n add a controlled phase gate
    for k in range(L):
        for j in range(L):
            phi = 2 * np.pi * A * 2**(k + j) / N
            qc.cp(phi, reg_m[k], reg_n[j])

### $F_A$ gate

A-scaled DFT operator on an $L$-qubit register
$$
F_2^L|k⟩ \mapsto \frac1{\sqrt{2^L}}\sum_{\ell=0}^{2^L - 1} e^{\frac{-2\pi i A k\ell}{2^L}}|\ell⟩ \tag{13}
$$

In [ ]:
def F(qc: QuantumCircuit,
             reg: QuantumRegister,
             A: int) -> None:
    L = len(reg)
    for i in range(L):
        qc.h(reg[i])
        for j in range(i + 1, L): 
            angle = 2 * np.pi * A / 2**(j - i + 1)
            qc.cp(-angle, reg[j], reg[i])
    for i in range(L // 2):
        qc.swap(reg[i], reg[L - 1 - i])

### $F_A^\dagger$ gate

Inverse A-scaled DFT, i.e. the Hermitian adjoint of `F`.

In [ ]:
def IF(qc: QuantumCircuit,
       reg: QuantumRegister,
       A: int
) -> None:
    L = len(reg)
    # Undo the bit-reversal swap
    for i in range(L // 2):
        qc.swap(reg[i], reg[L - 1 - i])
    # Reverse the controlled phase rotations
    for i in range(L - 1, -1, -1):
        for j in range(i + 1, L):
            angle = 2 * np.pi * A / 2**(j - i + 1)
            qc.cp(angle, reg[j], reg[i])
        qc.h(reg[i])

### Controlled-$W^{2^k}$ gate

Perform QFT-based cyclic increments $|x⟩\mapsto |x+2^k \bmod 2^L⟩$ for each `k` under control qubit `ctrl`.

In [ ]:
def W(qc: QuantumCircuit,
      ctrl, # Control qubit
      target: QuantumRegister
) -> None:
    L = len(target)
    for k in range(L):
        qc.append(QFT(L, do_swaps=True).to_gate(label="QFT"), target)
        for j in range(L):
            phi = 2 * np.pi * (2**k) * 2**j / (2**L)
            qc.cp(phi, ctrl, target[j])
        qc.append(QFT(L, inverse=True, do_swaps=True).to_gate(label="IQFT"), target)

### $K_G$ gate

Reset phase register to undo the phase estimation on the cyclic permutation $W$.

In [ ]:
def K(qc: QuantumCircuit,
      reg_m: QuantumRegister,
      reg_n: QuantumRegister,
      A: int
) -> None:
    L = len(reg_m)
    IF(qc, reg_m, A)
    for k in range(L):
        W(qc, reg_m[k], reg_n)
    qc.h(reg_m)
    for i in range(L):
        qc.swap(reg_m[i], reg_n[i])

### $U_A$ gate

The modular multiplication gate
$$
|m⟩|0⟩ \xrightarrow{U_A} |m⟩|mA \bmod N⟩
$$

In [ ]:
def U(A: int,
      N: int,
      L: int
): # -> Gate
    m = QuantumRegister(L, name = 'm')
    a = QuantumRegister(L, name = 'a')
    qc = QuantumCircuit(m, a, name = f"U")

    S(qc, a, N)
    V(qc, m, a, A, N)
    K(qc, m, a, A)
    
    S(qc, a, N)
    V(qc, m, a, 1, N)
    K(qc, m, a, 1)

    return qc.to_gate()

## Shor's Algorithm

Shor's algorithm finds the order $r$ of $A\bmod N$ by applying Quantum Phase Estimation on the modular multiplication unitary $U_A$.

In [ ]:
def circuit(N: int,
                 A: int,
                 t: int
) -> QuantumCircuit:
    L = ceil(log2(N))

    pe = QuantumRegister(t, 'pe')
    m  = QuantumRegister(L, 'm')
    n  = QuantumRegister(L, 'a')
    c  = ClassicalRegister(t, 'c')

    qc = QuantumCircuit(pe, m, n, c)
    qc.h(pe)
    qc.x(m[0])

    for k in range(t):
        A_pow = pow(A, 2**k, N)
        UA = U(A_pow, N, L).control(1)
        qc.append(UA, [pe[k]] + m[:] + n[:])
        
    qc.append(QFT(t, inverse=True, do_swaps=True).to_gate(label="IQFT"), pe)
    qc.measure(pe, c)
    return qc

### Order Extraction

Convert the most frequent bit-string measurement outcome `k` into a fraction $\frac{k}{2^t}\approx \frac{s}{r}$ and check whether $A^r \equiv 1 \mod N$.

In [ ]:
def order(counts: dict[str, int],
                  t: int,
                  N: int,
                  A: int
) -> int | None:
    freq = sorted(
        ((int(bs, 2), cnt) for bs, cnt in counts.items()),
        key = lambda x: -x[1])
    for k, _ in freq:
        if k == 0:
            continue
        frac = Fraction(k, 2**t).limit_denominator(N)
        r = frac.denominator
        if r > 1 and pow(A, r, N) == 1:
            return r
    return None

### Run Shor's Algorithm

Sweep over increasing phase estimation register sizes intil an even order `r` is found.

Please obtain your API key from [BlueQubit](https://app.bluequbit.io).

In [ ]:
def shor(N: int,
         A: int
) -> tuple[int, int] | None:
    for t in range(2, 2 * ceil(log2(N)) + 1):
        qc = circuit(N, A, t)
        bq = bluequbit.init("YOUR_TOKEN_HERE")
        result = bq.run(qc, device='cpu')
        counts = result.get_counts()
        r = order(counts, t, N, A)
        if r and r % 2 == 0:
            x = pow(A, r//2, N)
            f1 = gcd(x - 1, N)
            f2 = gcd(x + 1, N)
            if 1 < f1 < N and 1 < f2 < N:
                return f1, f2
    return None

### Example usage

Note: A-scaled DFT operator
 $$
 F^A_{2^L}|k\rangle\;=\;\frac{1}{\sqrt{2^L}}\sum_{\ell=0}^{2^L-1}e^{\frac{-2\pi i\,A k\ell}{2^L}}\lvert\ell\rangle
 $$
is **unitary** only when `A` is **odd**.

In [ ]:
if __name__ == "__main__":
    N, A = 15, 7
    print(f"Factoring N = {N} with A = {A}")
    result = shor(N, A)
    if result:
        print(f"✅️ Success: {N} = {result[0]} * {result[1]}")
    else:
        print("❌ Failed: try different A or larger t")


## 📚 References

[1] A. M. Patoary, A. Vikram, and V. Galitski, “A discrete Fourier transform based quantum circuit for modular multiplication in Shor’s algorithm,” Mar. 20, 2025, arXiv: arXiv:2503.10008. doi: 10.48550/arXiv.2503.10008.

[2] P. W. Shor, “Algorithms for quantum computation: discrete logarithms and factoring,” in Proceedings 35th Annual Symposium on Foundations of Computer Science, Santa Fe, NM, USA: IEEE Comput. Soc. Press, 1994, pp. 124–134. doi: 10.1109/SFCS.1994.365700.